In [23]:
from pyspark.sql import SparkSession

In [24]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [25]:
from pyspark.ml.regression import LinearRegression

In [26]:
training = spark.read.format(source='libsvm').load('sample_linear_regression_data.txt')
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [27]:
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')
lrModel = lr.fit(training)

In [28]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [29]:
lrModel.intercept

0.14228558260358093

In [30]:
training_summary = lrModel.summary
training_summary.r2

0.027839179518600154

In [31]:
training_summary.r2adj

0.007999162774081858

In [32]:
training_summary.rootMeanSquaredError

10.16309157133015

In [33]:
training_summary.pValues

[0.9927505031240562,
 0.30967074330990396,
 0.3178269194409711,
 0.003972477331573909,
 0.5201486327242175,
 0.16213017210149872,
 0.7102819001865635,
 0.5266812209137877,
 0.46256007153356316,
 0.37825808848978526,
 0.7592692146070568]

In [34]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
train_data,test_data = all_data.randomSplit([0.7,0.3])
train_data,test_data

(DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector])

In [35]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                338|
|   mean| 0.7980977307701485|
| stddev| 10.352331006454099|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [36]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                163|
|   mean|-0.8653725955866258|
| stddev| 10.186084649372049|
|    min|-28.046018037776633|
|    max| 24.290551295953957|
+-------+-------------------+



In [37]:
correct_model = lr.fit(train_data)

In [38]:
test_results = correct_model.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-27.418156907367205|
|-24.129873698003173|
|-28.530068493915213|
|-18.077998110521577|
|-21.652949437142652|
| -20.71766954607159|
| -19.15702011165671|
|-20.865975433000774|
|-20.164811955146742|
|-16.347337260140854|
| -18.26007349532096|
|-18.098157249938815|
|-13.221676518050916|
|-17.585649642296016|
| -16.93693721503734|
|-17.104842281881517|
| -15.61430793941045|
|-18.628524259206007|
|-18.020679669723318|
|-14.056438031677954|
+-------------------+
only showing top 20 rows



In [39]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-27.418156907367205|
|-24.129873698003173|
|-28.530068493915213|
|-18.077998110521577|
|-21.652949437142652|
| -20.71766954607159|
| -19.15702011165671|
|-20.865975433000774|
|-20.164811955146742|
|-16.347337260140854|
| -18.26007349532096|
|-18.098157249938815|
|-13.221676518050916|
|-17.585649642296016|
| -16.93693721503734|
|-17.104842281881517|
| -15.61430793941045|
|-18.628524259206007|
|-18.020679669723318|
|-14.056438031677954|
+-------------------+
only showing top 20 rows



In [40]:
correct_model = lr.fit(train_data)
test_result = correct_model.evaluate(test_data)
# fit and predicting model

In [41]:
test_result.rootMeanSquaredError

10.640414188503442

In [42]:
unlabelel_data = test_data.select('features')
unlabelel_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [43]:
predictions = correct_model.transform(unlabelel_data)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.6278611304094268|
|(10,[0,1,2,3,4,5,...|    0.64243357706666|
|(10,[0,1,2,3,4,5,...|  5.5802425577191395|
|(10,[0,1,2,3,4,5,...|  -4.759462306397763|
|(10,[0,1,2,3,4,5,...|  1.5954668213534402|
|(10,[0,1,2,3,4,5,...|  0.9349067564570522|
|(10,[0,1,2,3,4,5,...|-0.24531591855784413|
|(10,[0,1,2,3,4,5,...|  3.4373008620612664|
|(10,[0,1,2,3,4,5,...|   3.099412329270728|
|(10,[0,1,2,3,4,5,...|  -0.679155004068692|
|(10,[0,1,2,3,4,5,...|  1.5678664740098534|
|(10,[0,1,2,3,4,5,...|  1.9468078986617035|
|(10,[0,1,2,3,4,5,...| -2.7298360477436563|
|(10,[0,1,2,3,4,5,...|  1.7236403147254575|
|(10,[0,1,2,3,4,5,...|  1.2048489427980944|
|(10,[0,1,2,3,4,5,...|  1.3813266688329495|
|(10,[0,1,2,3,4,5,...|  0.2384502160981537|
|(10,[0,1,2,3,4,5,...|  3.2689793793733295|
|(10,[0,1,2,3,4,5,...|  2.6859121898009755|
|(10,[0,1,2,3,4,5,...| -0.706320